In [21]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time
import os

In [22]:
# 웹드라이버 준비
driver = webdriver.Chrome('./chromedriver_90')
## https://www.amazon.com/
url = 'https://www.amazon.com/Fire-HD-10/dp/B07K1RZWMC/ref=sr_1_5_sspa?dchild=1&keywords=fire+hd&qid=1624253358&sr=8-5-spons&psc=1&spLa=ZW5jcnlwdGVkUXVhbGlmaWVyPUEyQlJHNzcxWUI0RUJDJmVuY3J5cHRlZElkPUEwMTg2MTgyMVZUVUpaOU5CQlhaJmVuY3J5cHRlZEFkSWQ9QTA2MTA0MDAxWVU1TzJFVDdLQTBZJndpZGdldE5hbWU9c3BfbXRmJmFjdGlvbj1jbGlja1JlZGlyZWN0JmRvTm90TG9nQ2xpY2s9dHJ1ZQ=='
driver.get(url)

In [23]:
# ratings 클릭하여 스크롤 다운
sel_rate = driver.find_element_by_xpath('//*[@id="acrCustomerReviewText"]')
sel_rate.click()

In [24]:
# 전체 리뷰페이지
sel_review = driver.find_element_by_xpath('//*[@id="cr-pagination-footer-0"]/a')
sel_review.click()

In [59]:
# 컬럼 리스트 준비
review = []
user = []
score = []
review_date = []
loc = []
helpful = []

# 몇페이지까지 할지 결정
num = 5

for i in range(num) :
    # 현재 페이지에서 리뷰리스트 선택
    review_page = driver.page_source
    review_soup = BeautifulSoup(review_page, 'lxml')
    
    review_list = review_soup.find_all('div', class_='a-section celwidget')
    
    # 리뷰 리스트에서 하나씩 데이터 접근
    for j in review_list :
        # 리뷰
        temp_review = j.find('span', class_='a-size-base review-text review-text-content').text.strip()
        
        # 영상이 있을경우 필요없는 텍스트 제거
        check = 'Your browser does not support HTML5 video.'
        if check in temp_review :
            # \xa0는 ISO-8859-1로 인코딩 된 비 분리 공간입니다.
            check_idx = temp_review.rindex('\xa0')
            temp_review = temp_review[check_idx + 1:]
        review.append(temp_review)

        # 사용자
        temp_user = j.find('span', class_='a-profile-name').text
        user.append(temp_user)

        # 평점
        temp_score = j.find('span', class_='a-icon-alt').text[:3]
        score.append(temp_score)

        # 날짜
        temp_review_date = j.find('span', class_='a-size-base a-color-secondary review-date').text.split(' on ')[1]
        review_date.append(temp_review_date)

        # 지역
        temp_loc = j.find('span', class_='a-size-base a-color-secondary review-date').text.split(' on ')[0][16:]
        loc.append(temp_loc)

        # 도움
        # helpful 이 없느 경우 '0'으로 치환
        try :
            temp_helpful = j.find('span', class_='a-size-base a-color-tertiary cr-vote-text').text.split()[0]
        except AttributeError :
            temp_helpful = '0'
        helpful.append(temp_helpful)
        
    # 다음페이지 버튼 클릭 후 3초 딜레이
    next_btn = driver.find_element_by_xpath('//*[@id="cm_cr-pagination_bar"]/ul/li[2]/a')
    next_btn.click()
    time.sleep(3)
    
#     print(review[-1], user[-1], score[-1], review_date[-1], loc[-1], helpful[-1])

#     print(len(review), len(user), len(score), len(review_date), len(loc), len(helpful))

In [60]:
# 제품 이름으로 파일명 준비
item_name = 'Fire_HD_10_Tablet'
file_name = f'Amazon_review({item_name}).csv'

# DataFrame 생성
data_df = pd.DataFrame({'review' : review,
                        'user' : user,
                        'score' : score,
                        'review_date' : review_date,
                        'loc' : loc,
                        'helpful' : helpful})

# 생성된 DataFrame 확인
data_df.head()

# DataFrame -> CSV 파일로 출력
data_df.to_csv(file_name, index=False)

# 파일 저장여부 확인
if file_name in os.listdir() :
    print(f'{file_name} 저장 완료')
else : 
    print(f'{file_name} 파일이 없습니다.')


Amazon_review(Fire_HD_10_Tablet).csv 저장 완료


In [20]:
# 웹드라이버 종료
driver.close()